# Propagation d'une vidéo sur TikTok

## Fonctions web-scraping

Ces fonctions servent à tirer des musiques, themes et hashtags pour utiliser dans la modélisation

In [12]:
import requests
from bs4 import BeautifulSoup
import random

#Renvoie la liste de 50 musiques populaire actuelle dans l'odre (Spotify)

def musique():

	url = 'https://spotifycharts.com/viral/global/weekly/latest'

	response = requests.get(url)

	if response.ok :
		nom = []
		musique = []
		n = ''
		soup = BeautifulSoup(response.text, 'lxml')
		name = soup.findAll('strong')
	
		for strong in name :
			nom.append(str(strong))

		for i in nom :
			for s in range(8 , len(i) - 9):
				n = n + i[s]

			musique.append(n)
			n = ''

	return musique

#renvoie  la liste des hashtags tiktok populaire 


def hashtag():

	url = 'http://best-hashtags.com/hashtag/tiktok/'

	response = requests.get(url)

	if response.ok :
		nom = []
		hashtag =[]
		lr = []
		n = ''
		soup = BeautifulSoup(response.text, 'lxml')
		name = soup.findAll('p1')
	
		for p1 in name :
			nom.append(str(p1))

		for i in nom :
			for s in i :
				if (s == ' ') :
					lr.append(n)
					n = ''
				n +=s

	for j in range(1,len(lr)):
		hashtag.append(lr[j])


	return hashtag

# Renvoie la listes du top 21 des themes de video populaire sur Tiktok

def themes():

	url = 'https://influencermarketinghub.com/tiktok-video-ideas/'

	response = requests.get(url)

	if response.ok :
		lr = []
		l = []
		th = ''
		theme =[]

		soup = BeautifulSoup(response.text, 'lxml')
		name = soup.findAll('strong')
	
		for strong in name :
			lr.append(str(strong))

		for i in range(2,len(lr)) :
			l.append(lr[i])

		for j in l :
			for s in range(11 , len(j)-9 ) :
				th = th + j[s]

			theme.append(th)
			th = ''

	return theme

#Base video : Assigne de maniere aleatoire 1 theme , 1 musique , 1 hashtag au tupple (int,str,str,str)

def base_video(nb):

    l1 = themes()
    l2 = musique()
    l3 = hashtag()
    video = []
    i = 1 #Permet d'assigner un identifiant à chaque video

    while i < nb : 
	    video.append((i , random.choice(l1) , random.choice(l2) , random.choice(l3)))
	    i = i + 1

    return (video)

print("\n")
print(base_video(20))



[(1, 'Satisfying/Calming Videos', 'Disco Arranhado - Funk Remix', ' #funny'), (2, ' Life in Quarantine', 'Umrumda Değil', ' #instagram'), (3, 'Cute Animal Videos', 'Runaway', ' #fashion'), (4, 'Videos Based on Trending Hashtags', 'Ako Naman Muna', ' #funny'), (5, 'Social Media Challenges', 'Runaway', ' #tiktokindonesia'), (6, ' Videos of Your Daily Life', 'Tokyo', ' #art'), (7, ' Videos of Your Daily Life', 'I Hear a Symphony', ' #viral'), (8, ' Creative', 'Tokyo', ' #funny'), (9, 'Satisfying/Calming Videos', 'On The Ground', ' #youtube'), (10, ' Collaborations with Influencers', "Cupid's Chokehold / Breakfast in America", ' #youtube'), (11, 'Dance Videos', 'Disco Arranhado - Funk Remix', ' #fashion'), (12, ' Live Videos', 'Yucky Blucky Fruitcake', ' #explore'), (13, ' Live Videos', 'Tu Aake Dekhle', ' #youtube'), (14, 'Social Media Challenges', 'Leave The Door Open', ' #tiktokindia'), (15, ' Collaborations with Influencers', 'Iko Iko', ' #youtube'), (16, 'Videos Based on Trending Ha

## base_utilisateur

La fonction `base_utilisateur` renvoie un liste de tuples du format `identifiant, musique_favorie, hashtags_favoris, themes_favoris`. Sa sortie sera utilisée pour la fonction `interaction`.

In [3]:
def base_utilisateur(nb , long): #nb = nombre d'utilisateur , long = nb de donées dans chaque liste

	# renvoie une liste de Tuple de type (identifiant , liste musique favoris , liste hashtag favoris , liste theme favoris)
	base = []
	lh = [] #liste hashtag favoris
	lm = []	#liste musique favoris 
	lt = [] #liste theme favoris
	l1 = musique()
	l2 = hashtag()
	l3 = themes()
	j =  0
	i = 1

	while i < nb :
		while j < long :
			lm.append(random.choice(l1))
			lh.append(random.choice(l2))
			lt.append(random.choice(l3))
			j = j + 1
		
		base.append((i , lm , lh , lt))
		i = i + 1
		j = 0
		lm = []
		lh = []
		lt =[]

	return base

userbase = base_utilisateur(20,5)
videobase = base_video(20)

print("\n")
print(userbase)



[(1, ['Kiss (Never Let Me Go)', 'Gone', 'La Llevo Al Cielo', 'L-Gante: Bzrp Music Sessions, Vol.38', 'Friday (feat. Mufasa &amp; Hypeman) - Dopamine Re-Edit'], [' #tiktokers', ' #bhfyp', ' #funny', ' #photography', ' #youtube'], ['Cute Animal Videos', ' Makeover Videos', 'Cute Animal Videos', 'Dance Videos', 'Dialogue Reenactments\xa0']), (2, ['Tokyo', 'Leave The Door Open', 'Umrumda Değil', 'Dale Corte (Con Luxian, Keishon Trigga y S4vage)', 'Espirra o Lança'], [' #photography', ' #youtube', ' #followforfollowback', ' #bhfyp', ' #youtube'], [' Creative', ' Freeze-Frame Videos', ' Life in Quarantine', ' Freeze-Frame Videos', ' Makeover Videos']), (3, ['L-Gante: Bzrp Music Sessions, Vol.38', 'I Hear a Symphony', 'Iko Iko', 'Rasputin', 'Flamenco y Bachata'], [' #funny', ' #tiktokindia', ' #music', ' #tiktok', ' #tiktok'], ['Cute Animal Videos', ' Collaborations with Influencers', ' Voiceovers', 'Dialogue Reenactments\xa0', ' Food/Cooking Videos']), (4, ['Friday (feat. Mufasa &amp; Hype

## add_interaction

`add_interaction` sert à rajouter une intéraction à une vidéo, reliée à un utilisateur généré par `base_utilisateur`. Cette donnée sera gardée dans la liste `interactions` sous format d'un dictionnaire de tuples `identifiant, liked_videos, commented_videos, shared_videos`. Il sera mis en boucle par `tour_modele` 

In [10]:
def gen_user_pref(base_utilisateurs, base_videos, start_amount): # start_amount représente le nombre de vidéos au début
    b_u = base_utilisateurs.copy()
    b_v = base_videos.copy()
    videos_liked = [] #liste des videos likées
    videos_shared = [] #liste des vidéos commentées
    videos_commented = [] #liste des vidéos partagées
    for i in range(start_amount):  
        v = random.choice(b_v)
        print(v)
        if v not in videos_liked:
            videos_liked.append(v[0])
        v = random.choice(b_v)
        if v not in videos_commented:
            videos_commented.append(v[0])
        v = random.choice(b_v)
        if v not in videos_shared:
            videos_shared.append(v[0])
        b_u.append([videos_liked,videos_commented,videos_shared])
        
    return b_u

In [11]:
userprefs = gen_user_pref(userbase,videobase, 5)
#print(userprefs)

(11, 'Song Imitations', 'Flamenco y Bachata', ' #youtube')
(17, 'Cute Animal Videos', 'Iko Iko', ' #music')
(15, ' Makeover Videos', 'I Hear a Symphony', ' #tiktokindia')
(6, 'Social Media Challenges', 'Dale Corte (Con Luxian, Keishon Trigga y S4vage)', ' #love')
(4, ' Food/Cooking Videos', 'en el Refuego', ' #tiktokindonesia')
(13, ' Collaborations with Influencers', 'telepatía', ' #explorepage')
(12, 'Satisfying/Calming Videos', 'Tu Aake Dekhle', ' #explorepage')
(19, ' Voiceovers', 'telepatía', ' #photography')
(14, ' Science Experiments', 'To the Bone', ' #memes')
(13, ' Collaborations with Influencers', 'telepatía', ' #explorepage')
(16, 'Making Art', 'Fiel', ' #meme')
(15, ' Makeover Videos', 'I Hear a Symphony', ' #tiktokindia')
(12, 'Satisfying/Calming Videos', 'Tu Aake Dekhle', ' #explorepage')
(18, ' Fruit Cutting', '3KIPA', ' #love')
(4, ' Food/Cooking Videos', 'en el Refuego', ' #tiktokindonesia')
(13, ' Collaborations with Influencers', 'telepatía', ' #explorepage')
(2, ' 

In [6]:
interactions = dict()

In [7]:
#users own video for interactions

def add_interaction(base_utilisateurs, base_videos, interactions):
    if : # decide condition
        

IndentationError: expected an indented block (<ipython-input-7-28d4f803713b>, line 2)

# Résumé fonctionnement

`base_utilisateurs` et `base_videos` sont immutables.

`base_utilisateurs` commence avec des dictionnaires `ident:` tuples `(theme, musique, hashtag)`.
`base_videos` commence avec des dictionnaires `ident:` tuples `(theme, musique, hashtag)`.
(note : on pourrait peut-être utiliser la même fonction pour les créer avec des nombres d'itérations différents (ou même un lien entre le nb utilisateurs et le nb vidéos)

Stockage de données :

- `interactions` : dictionnaire `identifiant vidéo: [vue, like, comm, partage]`
- `interaction_utilisateur` de format `ident user : [ident video]

Comment on va itérer :

- on va parcourir la base de vidéos et la base d'utilisateurs pour accumuler des intéractions: (fonction `tour_modele`)
    - paramètres : `base_utilisateurs, base_videos (max_iter, on verra)`
    - on prend un utilisateur random de `base_utilisateurs`:
        - on prend des vidéos aléatoirement de `base_videos`
        - on compare aux préferences de l'utilisateur (il suffit qu'une soit présente) (fonction `compare_video`)
        - on continue jusqu'à qu'on trouve une vidéo qui correspond
            - on rajoute la vidéo à intéractions : (fonction `add_interaction)
                - on rajoute pour `interactions(ident)` une vue systématiquement
                - un like (aléatoirement)
                - un commentaire (aléatoirement)
                - un partage (aléatoirement)
            - on rajoute aussi la vidéo à `interaction_utilisateur`
- on décide un poids à attribuer à chaque intéraction (ce seront les paramètres d'une fonction)
- on utilise cela pour calculer un coefficient de popularité pour chaque vidéo (fonction `coefficient_popularite`)
- cela détermine ce qui est prioritisé par la modélisation de Manissa et Salwa

# compare_video

Compare les caractéristiques d'une vidéo aux préférences d'un utilisateur. Si elles sont les mêmes, True est retourné. Est utilisée dans `add_interaction`.

In [4]:
def compare_video(utilisateur, video): # iteration for videos will be done separately either on tour_modele or a function like compare_videos
    for i in range (1,len(utilisateur)):
        if utilisateur[i] == video[i]:
            return True
    return False

## Représentation graphique

Lorsque vous ouvrez le dossier, exécutez le fichier propagation_video_animation.py. Normalement, vous devez tomber sur une fenêtre tkinter qui demandera d'entrer les paramètres.
En fait, elles ne sont pas obligatoires, même si que les entrées sont vides et que vous cliquez sur "Start", ça fonctionnera.

Une fois que vous cliquez sur "Start", une nouvelle fenêtre apparaîtra, celle de pygame. Comme avant, vous pouvez utiliser espace, puis les touches directionnelles pour déplacer tiktok.
D'ailleurs, vous pouvez voir la confirmation des entrées dans la console normalement.

Comme avant, lorsque vous être en contact avec un humain, la console affiche le numéro de la collision, puis le nombre de vues augmente. Mais cette fois, le nombre de vues s'arrête à un moment précis, celui de l'objectif du nombre de vues.